In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [2]:
df_main = pd.read_csv(r'C:\Users\Prawin\Desktop\Praxis assessment\IML\Assignment-2\imports-85.data.csv')
df_main

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,...,c17,c18,c19,c20,c21,c22,c23,c24,c25,l1
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
201,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
202,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
203,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.4,23.0,106,4800,26,27,22470


In [3]:
df_main.columns

Index(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11',
       'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21',
       'c22', 'c23', 'c24', 'c25', 'l1'],
      dtype='object')

In [4]:
df_main.drop(['c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9','c15','c16','c18','c19','c20','c22','c23'],inplace=True,axis=1)

In [5]:
df_main = df_main.loc[df_main['l1']!='?']

In [23]:
df_main['l1'] = pd.to_numeric(df_main['l1'])

C:\Users\Prawin\AppData\Local\Temp/ipykernel_19884/3909061628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['l1'] = pd.to_numeric(df_main['l1'])


In [7]:
df_main.dropna(inplace=True)

C:\MyDrive\Term1\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [8]:
df_main.dropna(inplace=True)

In [16]:
def knn_method(df,df_y,df_test,k,target_var):
    df_main = pd.concat([df,df_y],axis = 1)
    master_dist=[]
    for y in range(0,df_test.shape[0]):
        distances=[]
        for x in range(0,df.shape[0]):
            res1 = 0
            a=(df.iloc[x,:]-df_test.iloc[y,:])**2
            distances.append(a.sum()**0.5)
        master_dist.append(distances)

    del_df=pd.DataFrame()    
    del_df['distances']=master_dist
    from scipy.stats import rankdata
    rank_list = []
    for x in del_df['distances']:
        rank_temp = rankdata(x,method='min')
        rank_list.append(rank_temp)

    del_df['rank']=rank_list

    df_y_pred=[]
    for a in range(df_test.shape[0]):
        temp_df = (df_main.set_index(keys=del_df['rank'][a])).sort_index()
        val = 0
        val = temp_df[target_var][:k].sum()
        val = val/k
        df_y_pred.append(val)
    df_test['y_pred'] = df_y_pred
    return df_test

In [20]:
def cv_method(cv,df_main,k=3):
    from sklearn.metrics import mean_absolute_percentage_error
    df_shuf = df_main.sample(frac=1, random_state=42)
    df_shuf = df_shuf.reset_index(drop=True)
    len_df = df_main.shape[0]
    train_mape_list = []
    test_mape_list = []

    for i in range(cv):
        start = int(i * len(df_shuf) / cv)
        end = int((i + 1) * len(df_shuf) /cv)
        train_x = pd.concat([df_shuf.iloc[:start,:-1], df_shuf.iloc[end:,:-1]])
        test_x = df_shuf.iloc[start:end,:-1]
        train_y = pd.concat([df_shuf.iloc[:start,-1], df_shuf.iloc[end:,-1]])
        test_y = df_shuf.iloc[start:end,-1]
        test_new = knn_method(train_x,train_y,test_x,k,'l1')
        test_mape = mean_absolute_percentage_error(test_y,test_new['y_pred'])
        test_mape_list.append(test_mape)
    return test_mape_list

In [21]:
cv_method(5,df_main)

[0.19089685026337014,
 0.16620699917762807,
 0.15696330055844998,
 0.12709033922259122,
 0.1884176448515598]